In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller

df = pd.read_csv("psl_cfx_2010_2016_complet.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date")
df["passages"].plot(figsize=(12,5))
plt.title("Activité hospitalière hebdomadaire")
plt.show()


KeyError: 'date'

In [ ]:
result = adfuller(df["passages"])
print("p-value:", result[1])


In [ ]:
SARIMA(p,d,q) (P,D,Q,52)
model = SARIMAX(
    df["passages"],
    order=(1,1,1),
    seasonal_order=(1,1,1,52)
)

result = model.fit()
print(result.summary())


In [ ]:
forecast = result.get_forecast(steps=26)
conf_int = forecast.conf_int()

plt.figure(figsize=(12,5))
plt.plot(df["passages"], label="Historique")
plt.plot(forecast.predicted_mean, label="Prévision")
plt.fill_between(conf_int.index,
                 conf_int.iloc[:,0],
                 conf_int.iloc[:,1],
                 alpha=0.3)
plt.legend()
plt.show()


In [ ]:
print("AIC:", result.aic)
print("BIC:", result.bic)
